# Importing the dependencies

In [ ]:
!nvidia-smi

Wed Jun 26 13:33:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install google-generativeai --upgrade -q
!pip install --default-timeout=100 future

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import sqlite3
import time
import torch

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax


import google.generativeai as genai

import os
from dotenv import load_dotenv


load_dotenv()
GOOGLE_API_KEY_MOVIE_RECOMMENDER = os.getenv("GOOGLE_API_KEY_MOVIE_RECOMMENDER")

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# Loading the Database
We will now load the database and work with some high end language models, and LLMs, to better understand the dataset and pre compute some of the features that we will use in the model.

In [4]:
data_query_based = pd.read_csv("../Data/Movies_Query_Based.csv")
data_query_based.head()

,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,poster_path,backdrop_path
0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...",/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg
1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...",/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg
2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg
3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...",/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...",/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg


# Sentiment Analysis with ROBERTA
As we have seen in the previous notebook [here](./Sentiment_Model.ipynb), the sentiment analysis done using ROBERTA has proved to be the most efficient to correctly predict the sentiment of the text. We will now use the same model to predict the sentiment of the text in the dataset, and store them as values.

In [ ]:
MOD = f"aychang/roberta-base-imdb"
tokenizer = AutoTokenizer.from_pretrained(MOD)
model = AutoModelForSequenceClassification.from_pretrained(MOD).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def sentiment_pred_roberta(text):
    encoded_text = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    output = model(**encoded_text)

    scores = output[0][0].detach().cpu().numpy()

    scores = softmax(scores)

    max_index = np.argmax(scores)

    if max_index == 0:
        sentiment = 'negative'
    elif max_index == 1:
        sentiment = 'positive'

    return sentiment

Now, in case there is any reviews list which is empty, we can go ahead and drop that row.

In [ ]:
for i in range(0,966):
  if(len(eval(data_query_based["reviews"].iloc[i]))==0):
    # If it has no reviews, we will drop it from the final data
    data_query_based = data_query_based.drop(i)

In [ ]:
data_query_based.shape

(966, 16)

In [ ]:
def sentiment_list_roberta(review_list):
  review_sentiment = []
  for review_text in review_list:
      review_sentiment.append(sentiment_pred_roberta(review_text))

  return review_sentiment

Now that the functions are defined, let us go ahead and fetch the sentiment list for each review.

In [ ]:
data_query_based["review_sentiment"] = data_query_based["reviews"].apply(lambda x: eval(x)).apply(lambda x: sentiment_list_roberta(x))

In [ ]:
data_query_based.head()

,Unnamed: 0,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,poster_path,backdrop_path,review_sentiment
0,0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...",/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg,"['positive', 'positive', 'positive', 'positive..."
1,1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...",/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg,"['negative', 'positive', 'positive', 'positive..."
2,2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,"['positive', 'positive', 'positive', 'positive..."
3,3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...",/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,"['positive', 'positive', 'positive', 'positive..."
4,4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...",/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg,"['positive', 'negative', 'negative', 'negative..."


# Collective Sentiment Summarization using Google Gemini Pro

# Prompt Generation

In [ ]:
def set_prompt(movie_title, movie_reviews):
  overall_review_summarisation_prompt = f"""You are a seasoned movie critic and insightful social analyst. Your task is to distill the collective sentiment of the audience from a set of reviews and provide a concise summary.
  \n\n
  Input:
  a. Movie Title: {movie_title}
  b. Reviews: \n\n

  {movie_reviews}

  \n\n
  Instructions:
  1. Sentiment Analysis:
      a. Carefully analyze the tone and content of each review to identify the underlying sentiment (positive, negative, or mixed).
      b. Consider both explicit statements and subtle cues to gauge the overall emotional response.
  \n
  2. Identify Key Themes and Aspects:
      a. Pinpoint the most frequently mentioned elements of the movie (e.g., plot, acting, characters, specific scenes, themes).
      b. Note both the aspects that resonated most strongly with viewers and those that elicited criticism or disappointment.
  \n
  3. Summarize Audience Consensus:
      a. Craft a concise summary (4-5 sentences maximum) that encapsulates the collective audience sentiment towards the movie.
      b. Clearly state whether the overall reaction is predominantly positive, negative, or a mix of both.
      c. Highlight the specific aspects that garnered the most praise and criticism, using illustrative examples or phrases from the reviews if appropriate.
      d. Avoid vague or generic statements. Focus on concrete details and observations drawn directly from the audience's own words.

  \n\n
  Example Output:
  Movie Title (Example): The Grand Adventure
  Audience reaction to The Grand Adventure is largely positive, with viewers captivated by its breathtaking visuals and thrilling action sequences. The film's humor also resonated well, although some found the plot predictable. While the lead actor's performance received acclaim, several reviewers felt the supporting characters were underdeveloped.

  \n\nImportant:
  a. Refrain from using quotation marks or any other framing devices. Simply present the summary text directly.
  b. Maintain a neutral and objective tone, avoiding any personal opinions or biases.
  """
  return overall_review_summarisation_prompt

Let us first create the a function to return the final prompt embedded with the movie title and movie reviews list mapped to their original entry in the dataframe. Since we would be needing both the title and the movie reviews as a list, we can use a tuple to append these both together and use it to generate the final prompt from the template defined above.

Once we have the final prompt for a given entry, we can store it in our database for now, as we wil be using it later to hit the LLM for it's summary.

In [ ]:
def get_final_prompt(data_row):
  title_and_reviews = (data_row["title"],eval(data_row["reviews"]))

  movie_title = title_and_reviews[0]
  movie_list = title_and_reviews[1]

  movie_reviews = "\n".join([f"\t{i+1}. {review}" for i, review in enumerate(movie_list)])

  final_prompt = set_prompt(movie_title, movie_reviews)
  return final_prompt


In [ ]:
data_query_based["prompt"] = data_query_based.apply(get_final_prompt, axis=1)

In [ ]:
data_query_based.head()

,Unnamed: 0,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,poster_path,backdrop_path,review_sentiment,prompt
0,0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...",/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...
1,1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...",/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg,"['negative', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...
2,2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...
3,3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...",/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...
4,4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...",/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg,"['positive', 'negative', 'negative', 'negative...",You are a seasoned movie critic and insightful...


## Hitting the Gemini API

In [ ]:
MAX_REQUESTS_PER_MINUTE = 14
WAIT_TIME_SECONDS = 60 / MAX_REQUESTS_PER_MINUTE


def get_gemini_response(prompt):
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')

        safe = [
            {
                "category": "HARM_CATEGORY_DANGEROUS",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]

        response = model.generate_content(
            [prompt],
            safety_settings=safe
        )

        time.sleep(WAIT_TIME_SECONDS)

        # Handling blocked response
        if(response.candidates[0].safety_ratings[1].blocked == True):
            print(f"Response blocked by safety system on category {print(response.candidates[0].safety_ratings[1].category)} for {print(response.candidates[0].safety_ratings[1].category)}")
            return ""

        else:
          return response.text


    except Exception as e:
        print(f"Error occurred during request: {e}")
        return ""

In [ ]:
data_query_based["review_summary"] = data_query_based["prompt"].apply(get_gemini_response)

Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range


ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2510.69ms


Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of range
Error occurred during request: list index (0) out of ran

In [ ]:
data_query_based.head(10)

,Unnamed: 0,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,poster_path,backdrop_path,review_sentiment,prompt,review_summary
0,0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...",/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...,Guardians of the Galaxy Vol. 2 elicited a gene...
1,1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...",/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg,"['negative', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...,Creed II elicits a mixed response from audienc...
2,2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...,Avengers: Infinity War elicits a generally pos...
3,3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...",/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...,Avengers: Endgame elicited a largely positive ...
4,4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...",/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg,"['positive', 'negative', 'negative', 'negative...",You are a seasoned movie critic and insightful...,The audience reaction to Fifty Shades Freed is...
5,5,354912,tt2380307,Coco,2017,Family Animation Fantasy Music Comedy Adventure,8.200,Anthony Gonzalez,Gael García Bernal,Benjamin Bratt,Alanna Ubach,Renée Victor,Lee Unkrich,"['afterlife', 'mexico', 'musician', 'death', '...","[""I'm no critic, but Coco is close to movie pe...",/gGEsBPAijhVUFoiNpgZXqRVWJt2.jpg,/askg3SMvhqEl4OL52YuvdtY40Yb.jpg,"['positive', 'positive', 'positive', 'positive...",You are a seasoned movie critic and insightful...,Coco overwhelmingly receives a positive recept...
6,6,537915,tt4126476,After,2019,Romance Drama,7.160,Josephine Langford,Hero Fiennes Tiffin,Shane Paul McGhie,Khadijha Red Thunder,Dylan Arnold,Jenny Gage,"['based on novel', 'man and woman share a bubb...",['Almost painful to sit through. Story could b...,/u3B2YKUjWABcxXZ6Nm9h10hLUbh.jpg,/997ToEZvF2Obp9zNZbY5ELVnmrW.jpg,"['negative', 'negative', 'negative', 'negative...",You are a seasoned movie critic and insightful...,
7,7,335988,tt3371366,Transformers: The Last Knight,2017,Action Adventure Sci-Fi,6.100,Mark Wahlberg,Anthony Hopkins,Josh Duhamel,Stanley Tucci,Laura Haddock,Michael Bay,"['stabbed in the chest', 'violence', 'based on...","[""Like all the other sequence, the visual effe...",/s5HQf2Gb3lIO2cRcFwNL9sn1o1o.jpg,/1n00NlOGRFZVs8coBxyZLm5l4EC.jpg,"['negative', 'positive', 'positive', 'negative...",You are a seasoned movie critic and insightful...,Audience reaction to Transfo

In [ ]:
empty_summaries = 0
for summary in data_query_based["review_summary"]:
  if(summary==""):
    empty_summaries += 1

print("The number of empty review summaries are : ",empty_summaries)

52

As we can see, a few entries genereated by the model are empty, so let us go ahead and remove those.

In [ ]:
data_query_based = data_query_based[data_query_based["review_summary"] != ""]
data_query_based.shape

(914, 20)

Let us now finally rearrange the columns.

In [ ]:
data_query_based = data_query_based.reindex(columns=['id','IMDB_ID', 'title','release_year' ,'genres','vote_average', 'Actor_1', 'Actor_2', 'Actor_3', 'Actor_4', 'Actor_5', 'Director', 'keywords', 'reviews','review_sentiment','review_summary','poster_path','backdrop_path'])
data_query_based.head()

,id,IMDB_ID,title,release_year,genres,vote_average,Actor_1,Actor_2,Actor_3,Actor_4,Actor_5,Director,keywords,reviews,review_sentiment,review_summary,poster_path,backdrop_path
0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,Adventure Action Sci-Fi,7.623,Chris Pratt,Zoe Saldaña,Dave Bautista,Vin Diesel,Bradley Cooper,James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...","['positive', 'positive', 'positive', 'positive...",Guardians of the Galaxy Vol. 2 elicited a gene...,/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg
1,480530,tt6343314,Creed II,2018,Drama,6.990,Michael B. Jordan,Sylvester Stallone,Tessa Thompson,Wood Harris,Russell Hornsby,Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...","['negative', 'positive', 'positive', 'positive...",Creed II elicits a mixed response from audienc...,/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg
2,299536,tt4154756,Avengers: Infinity War,2018,Adventure Action Sci-Fi,8.260,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson,Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...","['positive', 'positive', 'positive', 'positive...",Avengers: Infinity War elicits a generally pos...,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg
3,299534,tt4154796,Avengers: Endgame,2019,Adventure Sci-Fi Action,8.268,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Scarlett Johansson,Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...","['positive', 'positive', 'positive', 'positive...",Avengers: Endgame elicited a largely positive ...,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
4,337167,tt4477536,Fifty Shades Freed,2018,Drama Romance,6.699,Dakota Johnson,Jamie Dornan,Eric Johnson,Luke Grimes,Rita Ora,James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...","['positive', 'negative', 'negative', 'negative...",The audience reaction to Fifty Shades Freed is...,/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg


Let us now save the database to our database.

In [10]:
connection = sqlite3.connect("../SQL_Database/Movies.db")

data_query_based.to_sql('Movies_Query_Based', connection, if_exists='replace', index=False)

914

With this, we now have final queries databased ready, on which we can train the prompt to understand the queries and operate on the different types of queries.